In [1]:
from IPython.display import clear_output

In [2]:
from IPython.display import display_html

In [3]:
from IPython.display import Javascript

In [4]:
# define rooms and items

red_door = {
    "name": "red door",
    "type": "door",
}

green_door = {
    "name": "green door",
    "type": "door",
}

blue_door = {
    "name": "blue door",
    "type": "door",
}

yellow_door = {
    "name": "yellow door",
    "type": "door",
}

red_key = {
    "name": "key for red door",
    "type": "key",
    "target": red_door,
}

green_key = {
    "name": "key for green door",
    "type": "key",
    "target": green_door,
}

yellow_key = {
    "name": "key for yellow door",
    "type": "key",
    "target": yellow_door,
}

blue_key = {
    "name": "key for blue door",
    "type": "key",
    "target": blue_door,
}

couch = {
    "name": "couch",
    "type": "furniture",
}

piano = {
    "name": "piano",
    "type": "furniture",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

living_room = {
    "name": "living room",
    "type": "room",
}

outside = {
  "name": "outside"
}


all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [red_door, green_door, blue_door, yellow_door]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, red_door],
    "bedroom 1": [queen_bed, red_door, blue_door, green_door],
    "bedroom 2": [double_bed, dresser, green_door],
    "living room": [dining_table, yellow_door],
    "outside": [yellow_door],
    "piano": [red_key],
    "queen bed": [green_key],
    "double bed": [blue_key],
    "dresser": [yellow_key],
    "red door": [game_room, bedroom_1],
    "green door": [bedroom_1, bedroom_2],
    "blue door": [bedroom_1, living_room],
    "yellow door": [living_room, outside]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside,
    "steps": 0
}

In [7]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

def set_difficulty():
    """
    Prompts the player to choose the number of steps before the game ends"""
    print("Hello. Choose your preferred difficulty. The number of missteps allowed will depend on this")
    difficulty = input("Choose your difficulty: easy, normal, hard")
    clear_output()
    if difficulty == "easy":
        game_state["steps"] += 20
    elif difficulty == "normal":
        game_state["steps"] += 12
    elif difficulty == "hard":
        game_state["steps"] += 3
    else:
        set_difficulty()
    play_room(game_state["current_room"])
        
def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    set_difficulty()

def lose():
    """
    If the game is performed in more than 9 movements, you lose
    """
    print("Sorry, you took too long. You are now dead.")
    restart = input("Do you want to start again? [y/n]")
    if restart == "y":
        clear_output()
        restartkernel()
    elif restart == "n":
        print("Sorry to see you go")
    else:
        pass     
    

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    else:
        print("You are now in " + room["name"])
        items = [i["name"] for i in object_relations[room["name"]]]
        print("You explore the room. " + " You find: " + ", ".join(items))
        intended_action = input("What would you like to examine? Type the name of the item").strip().lower()
        clear_output()
        if game_state["steps"] == 0:
            lose()
        else:
            if intended_action in items:
                game_state["steps"] -=1
                examine_item(intended_action.strip())
            else:
                print("Not sure what you mean. What would you like to examine? Type the name of the item")
                play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
    
    if(next_room):
        play_room(next_room)
    else:
        play_room(current_room)


In [8]:
game_state = INIT_GAME_STATE.copy()

start_game()

Sorry, you took too long. You are now dead.


KeyboardInterrupt: 